In [ ]:
import glob

from netCDF4 import Dataset
from scipy.ndimage import rotate, maximum_filter

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
import meshplex
import optimesh
import subprocess

from scipy.interpolate import RectBivariateSpline

%matplotlib inline

In [ ]:
def jacobian(x0, y0, x1, y1, x2, y2):
    """
    jac = jacobian(x0, y0, x1, y1, x2, y2):
    calculates jac = det(M),
    where M is the matrix
    [[x1-x0, y1-y0], [x2-x0, y2-y0]].

    This is twice the area of a triangle with vertices:
    (x0, y0), (x1, y1), (x2, y2)

    Parameters:
    x0, x1, x2 (numpy arrays or floats) - x coords of the 3 points
    y0, y1, y2 (numpy arrays or floats) - y coords of the 3 points

    Returns:
    jac (same type as inputs)
    """
    return (x1-x0)*(y2-y0)-(x2-x0)*(y1-y0)


def get_areas(self):
    ''' Get areas of elements '''
    indices = self.indices

    areas = []
    for inds in indices:
        jac = self.get_jacobian(self.nodes_x, self.nodes_y, inds)
        areas.append( .5*np.abs(jac) )
    return areas

def get_jacobian(self, nodes_x, nodes_y, inds):
    """
    interface to calculate the jacobian for a single element
    self.get_jacobian(nodes_x,nodes_y,inds)

    Parameters:
    -----------
    nodes_x : numpy.ndarray
        x coordinates of mesh nodes
    nodes_y : numpy.ndarray
        y coordinates of mesh nodes
    inds : numpy.ndarray(int)
        - one row of self.indices
        - three elements
           - one for each node of the current triangular element
    """
    xy = []
    for n in inds:
        xy.append(nodes_x[n])
        xy.append(nodes_y[n])
    return nsl.jacobian(*xy)

In [ ]:
# load maximum concentration and average
idir = '/Data/sim/data/OSISAF_ice_conc_CDR/1979/03/'
ifiles = sorted(glob.glob(f'{idir}/*nc'))
c = np.dstack([Dataset(ifile)['ice_conc'][0].filled(np.nan) for ifile in ifiles])
c = np.nanmean(c, axis=2)

In [ ]:
ds = Dataset(ifiles[0])
xc = ds['xc'][:]
yc = ds['xc'][:]

In [ ]:
xlim = [140, 300]
ylim = [110, 310]

plt.figure(figsize=(10,10))
plt.imshow(c, interpolation='nearest')
plt.xticks(range(0,400,10))
plt.yticks(range(0,400,10))
plt.xlim(xlim)
plt.ylim(ylim[::-1])
plt.show()

In [ ]:
mask = np.isfinite(c).astype(int)
mask[:ylim[0]] = 0
mask[ylim[1]:] = 0
mask[:, :xlim[0]] = 0
mask[:, xlim[1]:] = 0
mask[170:290, :150] = 0
mask[185:290, :165] = 0
mask[194:290, :167] = 0
mask[208:290, :172] = 0
mask[220:290, :180] = 0
mask[220:290, :180] = 0
mask[230:290, :185] = 0
mask[245:255, :202] = 0

plt.figure(figsize=(10,10))
plt.imshow(mask, interpolation='nearest')

plt.xticks(range(0,400,10))
plt.yticks(range(0,400,10))
plt.xlim(xlim)
plt.ylim(ylim[::-1])
plt.show()

In [ ]:
mask2 = maximum_filter(mask, 7)
plt.figure(figsize=(10,10))
plt.imshow(mask2, interpolation='nearest')

plt.xticks(range(0,400,10))
plt.yticks(range(0,400,10))
plt.xlim(xlim)
plt.ylim(ylim[::-1])
plt.show()

In [ ]:
rows, cols = np.where(mask2)
xgrd, ygrd = np.meshgrid(xc, yc[::-1])
x0 = xgrd[rows, cols]
y0 = ygrd[rows, cols]
plt.figure(figsize=(10,10))
plt.plot(x0, y0, '.')
plt.show()
print(x0.size)

In [ ]:
x0 += np.random.randn(x0.size)
y0 += np.random.randn(y0.size)
t0 = Triangulation(x0, y0).triangles

def measure(x, y, t):
    area = .5*np.abs(jacobian(x[t][:,0], y[t][:,0], x[t][:,1], y[t][:,1], x[t][:,2], y[t][:,2]))
    dx = np.diff(np.hstack([x[t], x[t][:,0][None].T]))
    dy = np.diff(np.hstack([y[t], y[t][:,0][None].T]))
    edges = np.hypot(dx, dy)
    perim = edges.sum(axis=1)
    ap_ratio = area**0.5/ perim
    return area, edges, perim, ap_ratio


In [ ]:
a, e, p, r = measure(x0, y0, t0)
fig, ax = plt.subplots(1,4, figsize=(12,3))
ax[0].hist(a, 100, np.percentile(a, [1,99]))
ax[1].hist(e.flatten(), 100, np.percentile(e, [1,99]))
ax[2].hist(p, 100, np.percentile(p, [1,99]))
ax[3].hist(r, 100, np.percentile(r, [1,99]))
plt.show()

max_area = np.percentile(a, 99)
min_ap_ratio = 0.19
good_elems = (a < max_area) * (r > min_ap_ratio)
t0 = t0[good_elems]

a, e, p, r = measure(x0, y0, t0)
fig, ax = plt.subplots(1,4, figsize=(12,3))
ax[0].hist(a, 100, np.percentile(a, [0.1,99.9]))
ax[1].hist(e.flatten(), 100, np.percentile(e, [0.1,99.9]))
ax[2].hist(p, 100, np.percentile(p, [0.1,99.9]))
ax[3].hist(r, 100, np.percentile(r, [0.1,99.9]))
plt.show()


In [ ]:
plt.figure(figsize=(7,7))
plt.triplot(x0, y0, t0)
plt.xlim([1000,2000])
plt.ylim([-2000,-1000])
plt.show()

In [ ]:
# optimize mesh
p1, t1 = optimesh.optimize_points_cells(list(zip(x0,y0)), t0, "odt-fixed-point", 1.0e-5, 10)
x1, y1 = p1.T

In [ ]:
plt.figure(figsize=(7,7))
plt.triplot(x1, y1, t1)
plt.xlim([1000,2000])
plt.ylim([-2000,-1000])
plt.show()

In [ ]:
a, e, p, r = measure(x1, y1, t1)
fig, ax = plt.subplots(1,4, figsize=(12,3))
ax[0].hist(a, 100, np.percentile(a, [0.1,99.9]))
ax[1].hist(e.flatten(), 100, np.percentile(e, [0.1,99.9]))
ax[2].hist(p, 100, np.percentile(p, [0.1,99.9]))
ax[3].hist(r, 100, np.percentile(r, [0.1,99.9]))
plt.show()


In [ ]:
#mask[np.isnan(c)] = -1
plt.imshow(mask)

In [ ]:
rbs = RectBivariateSpline(xc, yc, mask[::-1], kx=1, ky=1)
m1 = rbs(y1, x1, grid=False)

In [ ]:
plt.figure(figsize=(5,5))
plt.triplot(x1, y1, t1)
scat = plt.scatter(x1,y1,10,m1)
plt.xlim([-2500,2500])
plt.ylim([-2500,2800])
plt.show()

In [ ]:
minm = 0.5
plt.figure(figsize=(15,15))
plt.triplot(x1, y1, t1)
scat = plt.scatter(x1,y1,10,m1>minm)
plt.xlim([-2500,0])
plt.ylim([-2500,0])
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
plt.triplot(x1, y1, t1)
scat = plt.scatter(x1,y1,10,m1>minm)
plt.xlim([0,2500])
plt.ylim([-2500,0])
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
plt.triplot(x1, y1, t1)
scat = plt.scatter(x1,y1,10,m1>minm)
plt.xlim([0,2500])
plt.ylim([0,2800])
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
plt.triplot(x1, y1, t1)
scat = plt.scatter(x1,y1,10,m1>minm)
plt.xlim([-2500,0])
plt.ylim([0,2800])
plt.show()

In [ ]:
np.savez('mesh_arctic_ease_25km_max7.npz', x=x1, y=y1, t=t1, mask=mask, xc=xc, yc=yc, landmask=np.isnan(c))